# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [17]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 2.496912393718343
 0.9626151861115573
 2.654577821087027

In [18]:
A

3×3 Matrix{Float64}:
 0.577689  0.967128  0.952095
 0.165205  0.543192  0.254218
 0.75958   0.990953  0.904045

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [19]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.217495  1.0       0.0
 0.760538  0.651492  1.0
U factor:
3×3 Matrix{Float64}:
 0.75958  0.990953  0.904045
 0.0      0.327665  0.0575924
 0.0      0.0       0.227014

In [20]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [21]:
Alu.P

3×3 Matrix{Float64}:
 0.0  0.0  1.0
 0.0  1.0  0.0
 1.0  0.0  0.0

In [22]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.217495  1.0       0.0
 0.760538  0.651492  1.0

In [23]:
Alu.U

3×3 Matrix{Float64}:
 0.75958  0.990953  0.904045
 0.0      0.327665  0.0575924
 0.0      0.0       0.227014

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [25]:
A\b

3-element Vector{Float64}:
 0.9999999999999988
 1.0
 1.0000000000000009

In [26]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999988
 1.0
 1.0000000000000009

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [27]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [28]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.596482  -0.301546  -0.743827
 -0.170579  -0.857944   0.484598
 -0.78429    0.415936   0.460311
R factor:
3×3 Matrix{Float64}:
 -0.968494  -1.44673   -1.32031
  0.0       -0.345489  -0.12918
  0.0        0.0       -0.168859

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [29]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.596482  -0.301546  -0.743827
 -0.170579  -0.857944   0.484598
 -0.78429    0.415936   0.460311

In [30]:
Aqr.R

3×3 Matrix{Float64}:
 -0.968494  -1.44673   -1.32031
  0.0       -0.345489  -0.12918
  0.0        0.0       -0.168859

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [31]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.2720359971026562
  0.17400256245579082
  4.147886021351139
vectors:
3×3 Matrix{Float64}:
 -0.804269   0.183277  -0.565297
  0.141315  -0.864982  -0.481494
  0.577218   0.467136  -0.669779

The values and the vectors can be extracted from the Eigen type by special indexing

In [32]:
AsymEig.values

3-element Vector{Float64}:
 -0.2720359971026562
  0.17400256245579082
  4.147886021351139

In [33]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.804269   0.183277  -0.565297
  0.141315  -0.864982  -0.481494
  0.577218   0.467136  -0.669779

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [34]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0          8.88178e-16   1.77636e-15
 -8.88178e-16  1.0          -8.88178e-16
  1.47105e-15  7.49401e-16   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [35]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [36]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [37]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-2.2467006475654223

In [38]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [39]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [40]:
@time eigvals(Asym);

  0.225827 seconds (268.63 k allocations: 25.896 MiB, 2.34% gc time, 73.94% compilation time)


In [41]:
@time eigvals(Asym_noisy);

  0.299313 seconds (14 allocations: 7.928 MiB)


In [42]:
@time eigvals(Asym_explicit);

  0.087893 seconds (68.34 k allocations: 12.645 MiB, 37.90% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.669902 seconds (823.18 k allocations: 229.291 MiB, 12.15% gc time, 26.00% compilation time)


6.7249639294089665

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 1//10  3//5   2//5
 1//5   1//10  1//5
 1//5   1//5   3//10

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 11//10
  1//2
  7//10

In [29]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  0//1   0//1
 1//2  1//1   0//1
 1//1  2//11  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1//5   1//10  1//5
 0//1  11//20  3//10
 0//1   0//1   1//22

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [ ]:
using LinearAlgebra

In [ ]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [ ]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [ ]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]